# install libraries

In [ ]:
! pip install transformers

     |████████████████████████████████| 1.9MB 7.4MB/s 
     |████████████████████████████████| 890kB 41.9MB/s 
     |████████████████████████████████| 3.2MB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=de609bd1a7ab847cd32d2b89d4a2ecdb0c593babb438e3bdf0ff26c8f0e7286f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# bert embedding

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
from transformers import BertTokenizer, TFBertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
input_ids=tf.constant([tokenizer.encode("how are you")])

In [ ]:
input_ids

<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 2129, 2024, 2017,  102]], dtype=int32)>

In [ ]:
outputs_feature_vector = model(input_ids)

In [ ]:
type(outputs_feature_vector[1])

tensorflow.python.framework.ops.EagerTensor

In [ ]:
tf.concat([outputs_feature_vector[1],pooled_ouput],axis=0)

TensorShape([3, 768])

In [ ]:
pooled_ouput=outputs_feature_vector[1]

In [ ]:
inputs = tf.keras.Input(shape=(768,))

In [ ]:
dense_1 = tf.keras.layers.Dense(50, activation=tf.nn.relu)(inputs)

In [ ]:
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(dense_1)

In [ ]:
my_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
my_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 768)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                38450     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
Total params: 38,552
Trainable params: 38,552
Non-trainable params: 0
_________________________________________________________________


In [ ]:
my_model.predict(outputs_feature_vector[1])

array([[0.15619561, 0.15370868, 0.5570506 , 0.13304512],
       [0.14415379, 0.14898092, 0.59028256, 0.11658277]], dtype=float32)

# q/a dataset

In [ ]:
! pwd

/content


In [ ]:
! wget http://jmcauley.ucsd.edu/data/amazon/qa/qa_Appliances.json.gz

--2021-02-28 05:50:56--  http://jmcauley.ucsd.edu/data/amazon/qa/qa_Appliances.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 975540 (953K) [application/x-gzip]
Saving to: ‘qa_Appliances.json.gz’

qa_Appliances.json. 100%[===================>] 952.68K  5.47MB/s    in 0.2s    

2021-02-28 05:50:56 (5.47 MB/s) - ‘qa_Appliances.json.gz’ saved [975540/975540]



In [ ]:
! ls

drive  qa_Appliances.json.gz  sample_data


In [ ]:
import gzip

In [ ]:
import pandas as pd

In [ ]:
import json

In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

In [ ]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, columns=["question","answer"],orient='index')

In [ ]:
df = getDF('/content/qa_Appliances.json.gz')

In [ ]:
y  = 0  for question
y = 1   for answer

In [ ]:
df.head()

,question,answer
0,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.
1,model number,This may help InSinkErator Model BADGER-1: Bad...
2,can I replace Badger 1 1/3 with a Badger 5 1/2...,Plumbing connections will vary with different ...
3,Does this come with power cord and dishwasher ...,It does not come with a power cord. It does co...
4,loud noise inside when turned on. sounds like ...,Check if you dropped something inside.Usually ...


In [ ]:
df['question']

0       I have a 9 year old Badger 1 that needs replac...
1                                            model number
2       can I replace Badger 1 1/3 with a Badger 5 1/2...
3       Does this come with power cord and dishwasher ...
4       loud noise inside when turned on. sounds like ...
                              ...                        
9006           How come it is compatible with all brands?
9007    Is it compatible to replace my Maytag UKF8001 ...
9008    I cannot find any official reference to testin...
9009    Can it be installed inside the refrigerator, a...
9010    I just got the filter in and it doesn't look l...
Name: question, Length: 9011, dtype: object

In [ ]:
y = np.zeros(df.shape[0]*2)

In [ ]:
y[df.shape[0]:]=1

In [ ]:
x_feature_embedding= tf.Variable(tf.zeros((len(y),768)))

In [ ]:
from tqdm import tqdm

In [ ]:
j=0
for i in tqdm(df['question']):
  input_ids=tf.constant([tokenizer.encode(i)])
  outputs_feature_vector = model(input_ids)
  pooler_output=outputs_feature_vector.pooler_output
  x_feature_embedding[j].assign(tf.reshape(pooler_output,[768,]))
  j=j+1


100%|██████████| 9011/9011 [38:43<00:00,  3.88it/s]


In [ ]:
j

10261

In [ ]:
j-9011

1250

In [ ]:
i

'Here are the compatible part numbers. You need to check your current filter number and see if it matches. The genuine OEM Whirlpool / KitchenAid 4396841 Refrigerator Ice & Water Filter replacement fits Whirlpool, KitchenAid, Maytag, Amana, Jenn-Air, and Kenmore side-by-side refrigerators with a push-button filter located in the grille at the bottom of the fridge. If you see the words "Fast Fill" or "PUR Measure" on your water dispenser, stick with this filter (4396841) because 4396841 has dual flow rates of 0.50 gpm and 0.85 gpm. If you don\'t see those words on your dispenser, you may upgrade to the newer, improved Whirlpool 4396710 filter. The newer 4396710 filter removes more contaminants than the older the 4396841 filter, including cysts and turbidity. The Whirlpool 4396841 refrigerator/icemaker water filter replaces the following older part numbers: Whirlpool / KitchenAid / Maytag / Amana / Jenn-Air 4396710, 4396710B, 4396710P, 4396710T 4396711, 4396711B, 4396711P, 4396711T 43968

In [ ]:
df['answer'][1250]

'Here are the compatible part numbers. You need to check your current filter number and see if it matches. The genuine OEM Whirlpool / KitchenAid 4396841 Refrigerator Ice & Water Filter replacement fits Whirlpool, KitchenAid, Maytag, Amana, Jenn-Air, and Kenmore side-by-side refrigerators with a push-button filter located in the grille at the bottom of the fridge. If you see the words "Fast Fill" or "PUR Measure" on your water dispenser, stick with this filter (4396841) because 4396841 has dual flow rates of 0.50 gpm and 0.85 gpm. If you don\'t see those words on your dispenser, you may upgrade to the newer, improved Whirlpool 4396710 filter. The newer 4396710 filter removes more contaminants than the older the 4396841 filter, including cysts and turbidity. The Whirlpool 4396841 refrigerator/icemaker water filter replaces the following older part numbers: Whirlpool / KitchenAid / Maytag / Amana / Jenn-Air 4396710, 4396710B, 4396710P, 4396710T 4396711, 4396711B, 4396711P, 4396711T 43968

In [ ]:
for i in tqdm(df['answer']):
  input_ids=tf.constant([tokenizer.encode(i)])
  outputs_feature_vector = model(input_ids)
  pooler_output=outputs_feature_vector.pooler_output
  x_feature_embedding[j].assign(tf.reshape(pooler_output,[768,]))
  j=j+1


 14%|█▍        | 1250/9011 [07:18<1:13:55,  1.75it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (651 > 512). Running this sequence through the model will result in indexing errors


InvalidArgumentError: ignored

In [ ]:
j

10352

In [ ]:
j=10261

In [ ]:
for i in tqdm(df['answer'][1251:]):
  try:
    input_ids=tf.constant([tokenizer.encode(i)])
    outputs_feature_vector = model(input_ids)
    pooler_output=outputs_feature_vector.pooler_output
    x_feature_embedding[j].assign(tf.reshape(pooler_output,[768,]))
  except Exception as e:
    print("exception",e)
    continue
  j=j+1

In [ ]:
y=tf.one_hot(y,2)


In [ ]:
y[:j].shape

TensorShape([18011, 2])

In [ ]:
j

18011

In [ ]:
len(y)

18022

In [ ]:
x_feature_embedding.shape

TensorShape([18022, 768])

In [ ]:
x_feature_embedding[:j].shape

TensorShape([18011, 768])



# training

In [ ]:
my_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 768)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                38450     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
Total params: 38,552
Trainable params: 38,552
Non-trainable params: 0
_________________________________________________________________


In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.01)
my_model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
my_model.fit(x_feature_embedding[:j],y[:j],batch_size=64,epochs=1,shuffle=True,validation_split=0.2)

226/226 [==============================] - 1s 3ms/step - loss: 0.8356 - accuracy: 0.7704 - val_loss: 0.3363 - val_accuracy: 0.8665


In [ ]:
my_model.fit(x_feature_embedding[:j],y[:j],batch_size=64,epochs=20,shuffle=True,validation_split=0.2)

Epoch 1/20
226/226 [==============================] - 1s 3ms/step - loss: 0.1793 - accuracy: 0.9323 - val_loss: 0.3223 - val_accuracy: 0.8704
Epoch 2/20
226/226 [==============================] - 1s 3ms/step - loss: 0.1584 - accuracy: 0.9400 - val_loss: 0.1049 - val_accuracy: 0.9664
Epoch 3/20
226/226 [==============================] - 1s 3ms/step - loss: 0.1527 - accuracy: 0.9412 - val_loss: 0.0583 - val_accuracy: 0.9820
Epoch 4/20
226/226 [==============================] - 1s 2ms/step - loss: 0.1428 - accuracy: 0.9452 - val_loss: 0.2352 - val_accuracy: 0.9098
Epoch 5/20
226/226 [==============================] - 1s 2ms/step - loss: 0.1376 - accuracy: 0.9480 - val_loss: 0.2723 - val_accuracy: 0.8934
Epoch 6/20
226/226 [==============================] - 1s 2ms/step - loss: 0.1384 - accuracy: 0.9482 - val_loss: 0.1786 - val_accuracy: 0.9378
Epoch 7/20
226/226 [==============================] - 1s 3ms/step - loss: 0.1311 - accuracy: 0.9489 - val_loss: 0.1439 - val_accuracy: 0.9528
Epoch 

In [ ]:
my_model.fit(x_feature_embedding[:j],y[:j],batch_size=64,epochs=30,shuffle=True,validation_split=0.2)

Epoch 1/30
226/226 [==============================] - 1s 3ms/step - loss: 0.1131 - accuracy: 0.9584 - val_loss: 0.0615 - val_accuracy: 0.9795
Epoch 2/30
226/226 [==============================] - 1s 3ms/step - loss: 0.1098 - accuracy: 0.9593 - val_loss: 0.1826 - val_accuracy: 0.9276
Epoch 3/30
226/226 [==============================] - 1s 2ms/step - loss: 0.1062 - accuracy: 0.9613 - val_loss: 0.2351 - val_accuracy: 0.9059
Epoch 4/30
226/226 [==============================] - 1s 3ms/step - loss: 0.1055 - accuracy: 0.9615 - val_loss: 0.1322 - val_accuracy: 0.9495
Epoch 5/30
226/226 [==============================] - 1s 3ms/step - loss: 0.1060 - accuracy: 0.9607 - val_loss: 0.0668 - val_accuracy: 0.9797
Epoch 6/30
226/226 [==============================] - 1s 2ms/step - loss: 0.1093 - accuracy: 0.9593 - val_loss: 0.1043 - val_accuracy: 0.9642
Epoch 7/30
226/226 [==============================] - 1s 2ms/step - loss: 0.1012 - accuracy: 0.9625 - val_loss: 0.2282 - val_accuracy: 0.8906
Epoch 

In [ ]:
my_model.fit(x_feature_embedding[:j],y[:j],batch_size=64,epochs=30,shuffle=True,validation_split=0.2)

Epoch 1/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0965 - accuracy: 0.9643 - val_loss: 0.0586 - val_accuracy: 0.9786
Epoch 2/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0925 - accuracy: 0.9654 - val_loss: 0.1339 - val_accuracy: 0.9525
Epoch 3/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0939 - accuracy: 0.9647 - val_loss: 0.0632 - val_accuracy: 0.9792
Epoch 4/30
226/226 [==============================] - 1s 2ms/step - loss: 0.1005 - accuracy: 0.9636 - val_loss: 0.1089 - val_accuracy: 0.9620
Epoch 5/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0914 - accuracy: 0.9679 - val_loss: 0.0965 - val_accuracy: 0.9639
Epoch 6/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0931 - accuracy: 0.9656 - val_loss: 0.0533 - val_accuracy: 0.9820
Epoch 7/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0916 - accuracy: 0.9661 - val_loss: 0.3388 - val_accuracy: 0.8659
Epoch 

In [ ]:
my_model.fit(x_feature_embedding[:j],y[:j],batch_size=64,epochs=30,shuffle=True,validation_split=0.2)

Epoch 1/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0906 - accuracy: 0.9668 - val_loss: 0.0756 - val_accuracy: 0.9722
Epoch 2/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0917 - accuracy: 0.9661 - val_loss: 0.0738 - val_accuracy: 0.9750
Epoch 3/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0903 - accuracy: 0.9663 - val_loss: 0.1317 - val_accuracy: 0.9561
Epoch 4/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0877 - accuracy: 0.9678 - val_loss: 0.1036 - val_accuracy: 0.9623
Epoch 5/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0910 - accuracy: 0.9654 - val_loss: 0.0533 - val_accuracy: 0.9792
Epoch 6/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0971 - accuracy: 0.9636 - val_loss: 0.1386 - val_accuracy: 0.9464
Epoch 7/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0840 - accuracy: 0.9692 - val_loss: 0.0750 - val_accuracy: 0.9725
Epoch 

In [ ]:
my_model.fit(x_feature_embedding[:j],y[:j],batch_size=64,epochs=30,shuffle=True,validation_split=0.2)

Epoch 1/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0860 - accuracy: 0.9688 - val_loss: 0.1112 - val_accuracy: 0.9600
Epoch 2/30
226/226 [==============================] - 1s 2ms/step - loss: 0.0745 - accuracy: 0.9739 - val_loss: 0.1108 - val_accuracy: 0.9598
Epoch 3/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0814 - accuracy: 0.9698 - val_loss: 0.1625 - val_accuracy: 0.9345
Epoch 4/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0840 - accuracy: 0.9684 - val_loss: 0.1969 - val_accuracy: 0.9178
Epoch 5/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0785 - accuracy: 0.9708 - val_loss: 0.1942 - val_accuracy: 0.9265
Epoch 6/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0800 - accuracy: 0.9707 - val_loss: 0.1474 - val_accuracy: 0.9498
Epoch 7/30
226/226 [==============================] - 1s 3ms/step - loss: 0.0784 - accuracy: 0.9713 - val_loss: 0.1341 - val_accuracy: 0.9512
Epoch 

In [ ]:
classes= {0:"question",1:"answer"}
def claasify_text(text):
  input_ids=tf.constant([tokenizer.encode(text)])
  outputs_feature_vector = model(input_ids)
  pooler_output=outputs_feature_vector.pooler_output
  result=my_model.predict(pooler_output)[0]
  return {"question":result[0],"answer":result[1]}


In [ ]:
claasify_text("is it bad product to buy")

{'answer': 0.007456115, 'question': 0.99254394}

In [ ]:
claasify_text("yes you should buy")

{'answer': 0.996779, 'question': 0.0032209468}

In [ ]:
my_model.save_weights("/content/drive/MyDrive/Colab Notebooks/saved_weight.h5")

In [ ]:
claasify_text("i buy this product but got defected one is there any replace policy")

{'answer': 0.00590668, 'question': 0.99409336}

In [ ]:
1)  bow
2)  tfidf
3) word embedding  (bert embedding)


In [ ]:
input_ids=tf.constant([tokenizer.encode("hi")])

In [ ]:
input_ids

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[ 101, 7632,  102]], dtype=int32)>